In [133]:
!pip install opendatasets
!pip install pandas
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 17.5 MB/s eta 0:00:00a 0:00:01


In [111]:
import opendatasets as od 
import pandas 

od.download( 
    "https://www.kaggle.com/datasets/msambare/fer2013/data") 


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013


In [14]:
import os

# List the files in the downloaded folder
dataset_path = "fer2013"
os.listdir(dataset_path)

['train', 'test']

In [15]:
import torch
import torch.nn as nn
from torch.optim import Adam

# Checking if CUDA is available
flag_cuda = torch.cuda.is_available()

if not flag_cuda:
    print('Using CPU')
else:
    print('Using GPU')


Using GPU


In [16]:

from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure single-channel grayscale
    transforms.Resize((48, 48)),  # Resize to match FER2013 dimensions
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor ✅
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values
])

valid_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure single-channel grayscale
    transforms.Resize((48, 48)),  # Resize to match FER2013 dimensions
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor ✅
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values
])

from torch.utils.data import DataLoader
from torchvision import datasets
# Load FER2013 dataset
# Load train and test datasets

train_dataset = datasets.ImageFolder(root="./fer2013/train", transform=transform)
valid_dataset = datasets.ImageFolder(root="./fer2013/test", transform=valid_transform)

In [17]:
# Dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [18]:
# Get dataset sizes
train_N = len(train_dataset)
valid_N = len(valid_dataset)

In [19]:

# Hyperparameters
n_classes = 7  # FER2013 has 7 emotions
IMG_CHS = 1  # Grayscale images
kernel_size = 3
flattened_img_size = 75 * 3 * 3

# Define the CNN Model
model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 48 x 48
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 24 x 24
    
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 24 x 24
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 12 x 12
    
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 12 x 12
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2, stride=2),  # 75 x 6 x 6
    
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(75 * 6 * 6, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)  # Output 7 classes
)


In [20]:
device = torch.device("cuda" if flag_cuda else "cpu")
model = model.to(device)


In [21]:
# Define Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [22]:
# Function to calculate batch accuracy
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [23]:
# Validation function
def validate():
    loss = 0
    accuracy = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    
    print(f'Valid - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')

In [24]:
# Training function
def train():
    loss = 0
    accuracy = 0
    model.train()
    
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    
    print(f'Train - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')


In [25]:
import torchmetrics
def compute_f1():
    model.eval()
    f1_metric = torchmetrics.F1Score(task="multiclass", num_classes=7).to("cuda")
    f1_metric.reset()
    
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)  # Keep everything on GPU
            output = model(x)
            predictions = torch.argmax(output, dim=1)

            f1_metric.update(predictions, y)  # Directly update the metric
    print(f"F1_score: {f1_metric.compute().item():.4f}")


In [26]:
# Training loop
epochs = 20

for epoch in range(epochs):
    print(f'Epoch: {epoch+1}/{epochs}')
    train()
    validate()
    compute_f1()

Epoch: 1/20
Train - Loss: 1411.3429 Accuracy: 0.3880
Valid - Loss: 318.1440 Accuracy: 0.4739
F1_score: 0.4739
Epoch: 2/20
Train - Loss: 1217.2645 Accuracy: 0.4795
Valid - Loss: 291.3671 Accuracy: 0.5056
F1_score: 0.5056
Epoch: 3/20
Train - Loss: 1130.8127 Accuracy: 0.5212
Valid - Loss: 280.9589 Accuracy: 0.5210
F1_score: 0.5210
Epoch: 4/20
Train - Loss: 1071.0096 Accuracy: 0.5439
Valid - Loss: 260.3144 Accuracy: 0.5553
F1_score: 0.5553
Epoch: 5/20
Train - Loss: 1023.9567 Accuracy: 0.5663
Valid - Loss: 263.5642 Accuracy: 0.5595
F1_score: 0.5595
Epoch: 6/20
Train - Loss: 987.3475 Accuracy: 0.5792
Valid - Loss: 257.5000 Accuracy: 0.5669
F1_score: 0.5669
Epoch: 7/20
Train - Loss: 949.0427 Accuracy: 0.5951
Valid - Loss: 255.8666 Accuracy: 0.5691
F1_score: 0.5691
Epoch: 8/20
Train - Loss: 922.3506 Accuracy: 0.6098
Valid - Loss: 247.6886 Accuracy: 0.5808
F1_score: 0.5808
Epoch: 9/20
Train - Loss: 884.5911 Accuracy: 0.6216
Valid - Loss: 249.6732 Accuracy: 0.5840
F1_score: 0.5840
Epoch: 10/20
T